In [1]:
import os
import yaml
from pathlib import Path
from textwrap import dedent
from typing import List, Dict
from dotenv import load_dotenv
import numpy as np
import pandas as pd
from crewai.process import Process
from crewai import Task, Agent, Crew
from langchain_openai import AzureChatOpenAI
from crewai.project import task, agent

load_dotenv()

True

In [2]:
root = Path().absolute().parent
config_dir = root / "src/config"


class MyAgents:
    def __init__(self, llm: AzureChatOpenAI) -> None:
        self.agents_config = yaml.safe_load(open(config_dir / "agents.yaml"))
        self.tasks_config = yaml.safe_load(open(config_dir / "tasks.yaml"))
        self.llm = llm

    @agent
    def phd_researcher(self) -> Agent:
        return Agent(
            config=self.agents_config["phd_researcher"],
            llm=self.llm,
        )
    
    @agent
    def academic_writer(self) -> Agent:
        return Agent(
            config=self.agents_config["academic_writer"],
            llm=self.llm,
        )
    
    @agent
    def reviewer_supervisor(self) -> Agent:
        return Agent(
            config=self.agents_config["reviewer_supervisor"],
            llm=self.llm,
        )
    

class MyTasks(MyAgents):
    @task
    def research(self) -> Task:
        return Task(
            config=self.tasks_config["research"],
            agent=self.phd_researcher(),
        )
    @task
    def writing(self) -> Task:
        return Task(
            config=self.tasks_config["writing"],
            agent=self.academic_writer(),
            context=[
                self.research()
            ]
        )
    @task
    def reviewing_and_final_draft(self) -> Task:
        return Task(
            config=self.tasks_config["reviewing_and_final_draft"],
            agent=self.reviewer_supervisor(),
            context=[
                self.research(),
                self.writing(),
            ]
        )

class MyCrew:
    def __init__(self) -> None:
        self.llm = self._create_llm()
        self.agents = self._create_agents()
        self.tasks = self._create_tasks()
        self.crew = self._create_crew()

    def _create_llm(self) -> AzureChatOpenAI:
        return AzureChatOpenAI(
            openai_api_type="azure",
            azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
            openai_api_version=os.getenv("AZURE_OPENAI_VERSION"),
            api_key=os.getenv("OPENAI_API_KEY"),
            azure_endpoint=os.getenv("OPENAI_ENDPOINT"),
            temperature=0.75,
        )


    def _create_agents(self) -> MyAgents:
        return MyAgents(self.llm)

    def _create_tasks(self) -> MyTasks:
        return MyTasks(self.llm)
    
    def _create_crew(self) -> Crew:
        return Crew(
            agents=[
                self.agents.phd_researcher(),
                self.agents.academic_writer(),
                self.agents.reviewer_supervisor(),
            ],
            process=Process.sequential,
            verbose=0,
        )

    def run(self, inputs: Dict) -> str:
        self.crew.tasks = [
            self.tasks.research(),
            self.tasks.writing(),
            self.tasks.reviewing_and_final_draft(),
        ]
        response = self.crew.kickoff(inputs=inputs)
        return response


In [3]:
crew = MyCrew()

In [4]:
inputs = {
    "research_topic": dedent(
        """\
            "Environmental Life Cycle Assessment (LCA) of Bioactives (such as fucoidan, alginate, etc.)
            from seaweed (especially brown seaweed) at laboratory scale and pilot scale" """,
    ),
    "research_paper_section": "Introduction",
}

In [5]:
response = crew.run(inputs=inputs)



> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer
Final Answer: my best complete final answer to the task.

---

Environmental Life Cycle Assessment (LCA) of Bioactives from Seaweed: Introduction

Seaweed, particularly brown seaweed, has gained significant attention in recent years for its potential as a sustainable source of bioactive compounds such as fucoidan and alginate. These bioactives possess a wide range of industrial, pharmaceutical, and nutraceutical applications due to their unique properties. However, to fully understand the environmental implications of producing these compounds, it is essential to conduct a comprehensive Environmental Life Cycle Assessment (LCA).

LCA is a systematic method used to evaluate the environmental impacts of a product or process throughout its entire life cycle, from raw material extraction to disposal. In the context of bioactives from brown seaweed, LCA enables researchers to assess the sustainability of the 

In [6]:
crew.crew.usage_metrics

{'total_tokens': 9871,
 'prompt_tokens': 5468,
 'completion_tokens': 4403,
 'successful_requests': 3}

In [8]:
print(response)

Seaweed, particularly brown seaweed, has garnered substantial interest in recent years as a sustainable source of bioactive compounds such as fucoidan and alginate. These bioactives exhibit a wide array of industrial, pharmaceutical, and nutraceutical applications due to their unique properties. However, a comprehensive Environmental Life Cycle Assessment (LCA) is crucial for understanding the environmental implications of producing these compounds.

LCA is a systematic method used to evaluate the environmental impacts of a product or process throughout its entire life cycle, from raw material extraction to disposal. In the context of bioactives from brown seaweed, LCA allows researchers to assess the sustainability of production processes, identify environmental impact hotspots, and propose improvements to reduce the ecological footprint.

Fucoidan and alginate are among the most studied bioactives extracted from brown seaweed. Fucoidan, a sulfated polysaccharide, is known for its ant